## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-14-22-49-19 +0000,nypost,Roaming pack of wild dogs terrorizes Texas cit...,https://nypost.com/2025/09/14/us-news/roaming-...
1,2025-09-14-22-40-47 +0000,startribune,Live: Vikings host the Falcons in 2025 home op...,https://www.startribune.com/minnesota-vikings-...
2,2025-09-14-22-39-25 +0000,nyt,"​Kim Seongmin, Defector Who Pierced North Kore...",https://www.nytimes.com/2025/09/14/world/asia/...
3,2025-09-14-22-38-31 +0000,nypost,NY Gov. Kathy Hochul expected to endorse Zohra...,https://nypost.com/2025/09/14/us-news/ny-gov-k...
4,2025-09-14-22-36-38 +0000,nypost,Utah gov decries ‘cancer’ of social media: ‘pl...,https://nypost.com/2025/09/14/us-news/utah-gov...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2459/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,22
51,kirk,21
50,charlie,17
68,new,8
148,up,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
77,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...,85
96,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...,78
43,2025-09-14-19-31-04 +0000,nypost,Eric Trump: Assassination of Charlie Kirk has ...,https://nypost.com/2025/09/14/us-news/eric-tru...,71
151,2025-09-14-04-20-55 +0000,nypost,Oxford Union president-elect will be disciplin...,https://nypost.com/2025/09/14/world-news/oxfor...,70
10,2025-09-14-22-14-00 +0000,wsj,Suspect in Charlie Kirk Shooting Isn’t Coopera...,https://www.wsj.com/us-news/suspect-in-charlie...,64


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
77,85,2025-09-14-15-50-46 +0000,nypost,Trump still furious over Charlie Kirk’s murder...,https://nypost.com/2025/09/14/us-news/utah-gov...
155,37,2025-09-14-03-00-00 +0000,wsj,Israeli Prime Minister Benjamin Netanyahu and ...,https://www.wsj.com/politics/national-security...
128,31,2025-09-14-09-30-00 +0000,wsj,In the weeks since President Trump’s 50% tarif...,https://www.wsj.com/economy/trade/global-trade...
102,31,2025-09-14-13-00-00 +0000,wsj,"As America’s 250th birthday approaches, Robert...",https://www.wsj.com/business/robert-reed-churc...
158,28,2025-09-14-02-56-48 +0000,nypost,Warner Bros. Discovery shares spike as CEO Dav...,https://nypost.com/2025/09/13/business/warner-...
65,26,2025-09-14-17-33-45 +0000,nyt,White House Seeks More Supreme Court Security ...,https://www.nytimes.com/2025/09/14/us/politics...
151,26,2025-09-14-04-20-55 +0000,nypost,Oxford Union president-elect will be disciplin...,https://nypost.com/2025/09/14/world-news/oxfor...
40,23,2025-09-14-19-40-06 +0000,nyt,Trump Administration Claims Vast Powers as It ...,https://www.nytimes.com/2025/09/14/us/politics...
96,23,2025-09-14-13-57-20 +0000,nypost,Charlie Kirk suspect Tyler Robinson not cooper...,https://nypost.com/2025/09/14/us-news/charlie-...
67,23,2025-09-14-17-29-00 +0000,nypost,Rubio visits Israel to smooth tensions after Q...,https://nypost.com/2025/09/14/world-news/rubio...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
